In [1]:
import random
import numpy as np
import torch

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cuda.deterministic = True

Скачиваем датасет

In [2]:
from torchvision.datasets import MNIST
from torchvision.transforms import transforms

grayscale_to_rgb = transforms.Compose([
    transforms.Resize(224),
    transforms.Grayscale(3),
    transforms.ToTensor(),
])

basic_transform = transforms.Compose([
    grayscale_to_rgb
])

train_dataset = MNIST(
    './MNIST/train', 
    download = True, 
    train = True,
    transform=basic_transform)
test_dataset = MNIST(
    './MNIST/test', 
    download = True, 
    train = False,
    transform=grayscale_to_rgb)

# train_ids = torch.arange(1_000)
# test_ids = torch.arange(100)

# train_dataset = torch.utils.data.Subset(train_dataset, train_ids)
# test_dataset = torch.utils.data.Subset(test_dataset, test_ids)

In [3]:
from torch.utils.data import DataLoader

dataloaders = {
    'train': DataLoader(
        dataset=train_dataset,
        batch_size=128,
        shuffle=True,
    ),
    'val': DataLoader(
        dataset=test_dataset,
        batch_size=128,
        shuffle=True,
    ),
}

In [4]:
from torch import nn
from torchvision import models

class MNISTClassifier(nn.Module):
    def __init__(self):
        super(MNISTClassifier, self).__init__()
        self.model = models.alexnet(pretrained=True)
        
        for param in self.model.parameters():
            param.required_grad = False
            
        self.fc1 = nn.Sequential(
            nn.Linear(1000, 64),
            nn.ReLU()
        )
        
        self.fc2 = nn.Sequential(
            nn.Dropout(0.25),
            nn.Linear(64, 10)
        )
    
    def embed(self, x):
        return self.fc1(self.model(x))
    
    def forward(self, x):
        alexnet_out = self.embed(x)
        return self.fc2(alexnet_out)

In [5]:
model = MNISTClassifier()

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), amsgrad=True, lr=1e-4)

In [6]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/MNIST')

In [7]:
import torchvision

images, labels = next(iter(dataloaders['train']))

img_grid = torchvision.utils.make_grid(images)
writer.add_image('MNIST', img_grid)

writer.add_graph(model, images)

In [8]:
from tqdm.notebook import tqdm, trange

EPOCH_NUM = 4

model = model.to('cuda')
i = 0
    
for epoch in trange(EPOCH_NUM, desc='Whole pipeline'):

    for phase in ['train', 'val']:
        dataloader = dataloaders[phase]
        
        if phase == 'train':
            model.train() 
        elif phase == 'val':
            model.eval()
        
        running_loss = 0.
        running_acc  = 0.
        
        for (X_batch, y_batch) in tqdm(dataloader, desc=f'Epoch: {epoch + 1}. Phase: {phase}'):
            optimizer.zero_grad()
        
            X_batch = X_batch.to('cuda')
            y_batch = y_batch.to('cuda')
        
            with torch.set_grad_enabled(phase == 'train'):
                y_pred = model(X_batch)
                
                loss_value = loss_function(y_pred, y_batch)
                y_pred_class = y_pred.argmax(dim=1)
                
                if phase == 'train':
                    loss_value.backward()
                    optimizer.step()
                   
            running_loss += loss_value.item()
            running_acc += (y_pred_class == y_batch.data).float().mean().data.cpu().numpy()
        
            N = 20
            if i % N == N - 1:
                writer.add_scalar('loss', running_loss / N, i)
                running_loss = 0.

                writer.add_scalar('accuracy', running_acc / N, i)
                running_acc = 0.
            i += 1

Whole pipeline:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch: 1. Phase: train:   0%|          | 0/469 [00:00<?, ?it/s]

Epoch: 1. Phase: val:   0%|          | 0/79 [00:00<?, ?it/s]

Epoch: 2. Phase: train:   0%|          | 0/469 [00:00<?, ?it/s]

Epoch: 2. Phase: val:   0%|          | 0/79 [00:00<?, ?it/s]

Epoch: 3. Phase: train:   0%|          | 0/469 [00:00<?, ?it/s]

Epoch: 3. Phase: val:   0%|          | 0/79 [00:00<?, ?it/s]

Epoch: 4. Phase: train:   0%|          | 0/469 [00:00<?, ?it/s]

Epoch: 4. Phase: val:   0%|          | 0/79 [00:00<?, ?it/s]

In [9]:
images, labels = next(iter(DataLoader(test_dataset, batch_size=256, shuffle=True)))

images = images.to('cuda')
writer.add_embedding(
    model.embed(images),
    metadata=[str(i) for i in labels],
    label_img=images)

In [10]:
writer.close()